<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color: skyblue">
    TIME SERIES DATA PROCESSING
    <br>
    CROSS BORDER FLOWS
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    Raw Data Downloading Notebook
</div>
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Each part of the following script was used to download the raw data for the Cross Border Flows Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Importing needed libraries
</div>

In [1]:
import pandas as pd
from entsoe import EntsoePandasClient
from entsoe.exceptions import NoMatchingDataError  # Ensure this line is included
from entsoe.exceptions import NoMatchingDataError, InvalidBusinessParameterError  # Import relevant exceptions
import os
import csv
from datetime import datetime
import pytz
from pytz import timezone, utc
import shutil

 <div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    2. ENTSO-E RESTful API.
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Connecting with the ENTSO-E API Tool.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
To donwload the neeeded data using the API tool, is mandatory to use a token autentication to connect and make the future request of data.
</div>

In [2]:
client = EntsoePandasClient(api_key='61e5bbbb-7e80-4540-a471-bd993873aa74')

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    3. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [3]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    4. Usefull Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
Indicating the year of all data is referring to in the variable data_year.
<br>
The universal_standar_time variable is going to be used to download all the time series data in this horary zone. Additionally as each european country belongs a particular time sector the corresponding time series data related to its time sector are going to be downloaded as well but in a different file.
<br>
Additionally there are some default parameters that has to be defined to the correct working and calling to the ENTSO-E downloading functions.
</div>

In [4]:
# Year to which data refers to:
data_year = 2022
data_year_1 = data_year + 1

In [5]:
# Additional string to be appended
additional_path = "/RawData/CrossBorderFlows/"
additional_path_1 = "/RawData/CrossBorderFlows/Raw_Data_Sources/"

# Construct the Outage_Factors_folder_path variable
cross_border_flows_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Outage_Factors_Raw_Data_folder_path variable
cross_border_flows_raw_data_folder_path = dispaSET_unleash_folder_path + additional_path_1

print("cross_border_flows_folder_path:", cross_border_flows_folder_path)
print("cross_border_flows_raw_data_folder_path:", cross_border_flows_raw_data_folder_path)

cross_border_flows_folder_path: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/
cross_border_flows_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/


In [6]:
# Define other parameters
type_marketagreement_type = 'A01'
contract_marketagreement_type = "A01"
process_type = 'A51'

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    5. Country List Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Defining the list of countries according to the available data. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
Just those countries that interchange flows with other countries different of the ones modelled in Dispa-SET are defined in the list.
</div>

In [7]:
# Define a list of country codes
cross_border_flows_per_unit_country_list = ["AL",
                                            "AT",
                                            "BA",
                                            "BE",
                                            "BG",
                                            "BY",
                                            "CH",
                                            "CY",
                                            "CZ",
                                            "DE",
                                            "DK",
                                            "EE",
                                            "ES",
                                            "FI",
                                            "FR",
                                            "GR",
                                            "HR",
                                            "HU",
                                            "IE",
                                            "IT",
                                            "LT",
                                            "LU",
                                            "LV",
                                            "MD",
                                            "ME",
                                            "MK",
                                            "MT",
                                            "NL",
                                            "NO",
                                            "PL",
                                            "PT",
                                            "RO",
                                            "RU",
                                            "RS",
                                            "SE",
                                            "SI",
                                            "SK",
                                            "TR",
                                            "UA",
                                            "UK"
                                           ]

In [8]:
# Define the directory and file path
file_name = 'country_list.csv'
file_path = os.path.join(cross_border_flows_raw_data_folder_path, file_name)

# Ensure the directory exists
os.makedirs(cross_border_flows_raw_data_folder_path, exist_ok=True)

# Create a DataFrame
df = pd.DataFrame(cross_border_flows_per_unit_country_list, columns=['Country_From'])

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

print(f"DataFrame saved to '{file_path}'")
cross_border_flows_country_list_file = file_path

DataFrame saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/country_list.csv'


<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [9]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"cross_border_flows_folder_path:                            {cross_border_flows_folder_path}")   
print (f"cross_border_flows_raw_data_folder_path:                   {cross_border_flows_raw_data_folder_path}")
print (f"cross_border_flows_country_list_file:                      {cross_border_flows_country_list_file}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2022
cross_border_flows_folder_path:                            /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/
cross_border_flows_raw_data_folder_path:                   /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/
cross_border_flows_country_list_file:                      /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/country_list.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Defining the sub-folders where all the cross border flows raw data is saved. 
</div>

In [10]:
# Convert data_year to string if it's not already
data_year = str(data_year)

# Read the CSV file into a DataFrame
df = pd.read_csv(cross_border_flows_country_list_file)

# Ensure the column 'Country_From' exists
if 'Country_From' not in df.columns:
    raise ValueError("Column 'Country_From' does not exist in the CSV file")

# Define the base directory where subfolders will be created
base_directory = os.path.join(cross_border_flows_raw_data_folder_path, data_year)

# Create a list to hold the paths of the created subfolders
country_folder_paths = []

# Create subfolders and save their paths
for country in df['Country_From']:
    # Create the subfolder path
    subfolder_path = os.path.join(base_directory, country)
    
    # Create the subfolder if it doesn't exist
    os.makedirs(subfolder_path, exist_ok=True)
    
    # Append the subfolder path to the list
    country_folder_paths.append(subfolder_path)

# Add the new column 'Country_Folder' to the DataFrame
df['Country_Folder'] = country_folder_paths

# Save the updated DataFrame back to the CSV file
df.to_csv(cross_border_flows_country_list_file, index=False)

print(f"Updated CSV file saved to '{cross_border_flows_country_list_file}' with new subfolders created.")

Updated CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/country_list.csv' with new subfolders created.


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Defining the neighbor countries. 
</div>

In [11]:
import pandas as pd

# Raw data as a multiline string
data = """
GR
CZ, DE, HU, IT, SI, CH
HR
FR, DE, LU, NL, UK
GR, RO, MK, RS, TR
LT
AT, FR, DE, IT

AT, DE, PL, SK
AT, BE, CZ, DK, FR, LU, NL, NO, PL, SE, CH
DE, NL, NO, SE, UK
FI, LV, RU
FR, PT
EE, NO, SE, RU
BE, DE, IT, ES, CH, UK
BG, IT, AL, MK, TR
HU, SI, BA, RS
AT, HR, RO, SK, SI, RS, UA
UK
AT, FR, GR, MT, SI, CH, MT, ME
LV, PL, SE, BY, RU
BE, DE
EE, LT, RU
RO
IT
BG, GR
IT
BE, DK, DE, NO, UK
DK, FI, DE, NL, SE, UK
CZ, DE, LT, SK, SE, UA
ES
BG, HU, MD, RS, UA
EE, FI, LV, LT
BG, HR, HU, RO
DK, FI, DE, LT, NO, PL
AT, HR, HU, IT
CZ, HU, PL, UA
BG, GR
HU, PL, RO, SK
BE, DK, FR, IE, NL, NO
"""

# Split the data into lines
lines = data.strip().split("\n")

# Initialize a list to hold the data
data_list = []

# Process each line
for line in lines:
    if line.strip() == "":
        # If the line is empty, add an empty list
        data_list.append([""] * 4)
    else:
        neighbors = line.split(", ")
        # Ensure the neighbors list has exactly 4 items
        neighbors = neighbors + [""] * (4 - len(neighbors))  # Fill with empty strings if less than 4
        neighbors = neighbors[:4]  # Truncate if more than 4
        data_list.append(neighbors)

# Create the DataFrame for new data
new_df = pd.DataFrame(data_list, columns=[f"Neighbor_{i}" for i in range(1, 5)])

# Path to the existing CSV file
existing_csv_file_path = cross_border_flows_country_list_file  # Update this path

# Read the existing CSV file into a DataFrame
existing_df = pd.read_csv(existing_csv_file_path)

# Concatenate the existing DataFrame with the new DataFrame horizontally
combined_df = pd.concat([existing_df, new_df], axis=1)

# Save the combined DataFrame back to the CSV file
combined_df.to_csv(existing_csv_file_path, index=False)

print(f"Data appended to '{existing_csv_file_path}' as new columns")


Data appended to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/country_list.csv' as new columns


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;color:skyblue">
    6. Raw Data Download
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Donwloading the cross border flows raw data. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
The crows border flow data is downloaded in separate files for each country.
<br>
Since the Acronym of Grece in the downloaded data is 'GR' and the Dispa-SET format for the country is 'EL'. All the needed changes in the used variables are done.
<br>
The downloaded files will be joined into a single csv file under the name of the country which the flow comes from.
<br>
The headers of these joined csv files are changed accordign the Dispa-SET cross border flow data format e.g. BE -> DE
</div>

In [12]:
# Define the start and end timestamps using the data_year variable
start = pd.Timestamp(f'{data_year}0101', tz='Europe/Brussels')
end = pd.Timestamp(f'{data_year_1}0101', tz='Europe/Brussels')

# Read the CSV file into a DataFrame
df = pd.read_csv(cross_border_flows_country_list_file)

# Ensure the necessary columns exist
required_columns = ['Country_From', 'Country_Folder'] + [f'Neighbor_{i}' for i in range(1, 5)]
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' does not exist in the CSV file")

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    country_code_from = row['Country_From']
    country_folder = row['Country_Folder']
    
    # Create the directory if it does not exist
    os.makedirs(country_folder, exist_ok=True)
    
    # Iterate through each neighbor column
    for neighbor_col in [f'Neighbor_{i}' for i in range(1, 5)]:
        country_code_to = row[neighbor_col]
        
        # Skip if the neighbor field is empty
        if pd.isna(country_code_to):
            continue
        
        try:
            # Query crossborder flows
            net_transfer = client.query_crossborder_flows(
                country_code_from, country_code_to, start=start, end=end
            )
            
            # Convert the index to a column
            net_transfer = net_transfer.reset_index()
            
            # Define the output file path
            output_file = os.path.join(country_folder, f'{country_code_to}.csv')
            
            # Save the DataFrame to a CSV file, including the index as a column
            net_transfer.to_csv(output_file, index=False)

            print(f"Data for {country_code_from} to {country_code_to} saved to '{output_file}'")

        except InvalidBusinessParameterError:
            print(f"Invalid business parameter for {country_code_from} to {country_code_to}. This pair may not be supported by the API.")
        
        except NoMatchingDataError:
            print(f"No matching data for {country_code_from} to {country_code_to} for the period {start} to {end}. Skipping.")
        
        except Exception as e:
            print(f"An error occurred while processing data for {country_code_from} to {country_code_to}: {e}")

print("All data has been processed and saved.")

Data for AL to GR saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/AL/GR.csv'
Data for AT to CZ saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/AT/CZ.csv'
Data for AT to DE saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/AT/DE.csv'
Data for AT to HU saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/AT/HU.csv'
Data for AT to IT saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/AT/IT.csv'
Data for BA to HR saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/BA/HR.csv'
Data for BE to FR saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/BE/FR.csv'
Data for BE to DE saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/BE/DE.csv'
Data for BE to LU saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Dat

Connection Error, retrying in 0 seconds


Data for TR to GR saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/TR/GR.csv'
An error occurred while processing data for UA to HU: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Data for UA to PL saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/UA/PL.csv'
Data for UA to RO saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/UA/RO.csv'
Data for UA to SK saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/UA/SK.csv'
Data for UK to BE saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/UK/BE.csv'
No matching data for UK to DK for the period 2022-01-01 00:00:00+01:00 to 2023-01-01 00:00:00+01:00. Skipping.
Data for UK to FR saved to '/home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/2022/UK/FR.csv'
Data for UK to IE saved to '/home/ray/Dispa-SET_Unleash/Ra

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
Cleaning folders. 
</div>

In [13]:
def erase_csv_file(file_path):
  """Erases a CSV file.

  Args:
    file_path: The path to the CSV file.
  """

  if os.path.exists(file_path):
    os.remove(file_path)
    print(f"File erased: {file_path}")
  else:
    print(f"File does not exist: {file_path}")

# Example usage:
file_path = cross_border_flows_country_list_file
erase_csv_file(file_path)

File erased: /home/ray/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources/country_list.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;color:skyblue">
The Cros Border Flows Raw Data is stored in the path. 
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;color:skyblue">
/Local/Path/to/Dispa-SET_Unleash/RawData/CrossBorderFlows/Raw_Data_Sources
</div>